In [ ]:
# !pip install -r ./requirements.txt

In [2]:
# !pip uninstall virny -y

In [4]:
# !pip install virny==0.4.0

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../../../..")

print('Current location: ', os.getcwd())

Current location:  /home/dh3553/projects/fairness-variance


## Import dependencies

In [4]:
import copy

from virny.utils.custom_initializers import create_config_obj
from virny.datasets import ACSIncomeDataset

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS
from configs.models_config_for_tuning import get_folktables_employment_models_params_for_tuning

from source.experiment_interface import run_exp_iter_with_disparate_impact

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'


## Define Input Variables

In [5]:
ROOT_DIR = os.getcwd()
META_EXPERIMENT_NAME = 'diff_bootstrap_sizes_exp'
EXPERIMENT_NAME = 'Baseline_acs_income'
BOOTSTRAP_SIZE = '10'
DB_COLLECTION_NAME = 'one_repair_lvl_many_models'
FAIR_INTERVENTION_PARAMS_LST = [0.0]
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', META_EXPERIMENT_NAME, EXPERIMENT_NAME, BOOTSTRAP_SIZE)

config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', META_EXPERIMENT_NAME,
                                EXPERIMENT_NAME, BOOTSTRAP_SIZE, 'folk_CA_2018_config.yaml')
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

## Define a db writer and custom fields to insert into your database

In [6]:
import os
from dotenv import load_dotenv

load_dotenv('./configs/secrets.env')
os.getenv("DB_NAME")

'fairness_variance'

In [7]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [8]:
import uuid

custom_table_fields_dct = {
#     'session_uuid': str(uuid.uuid4()),
    'session_uuid': '0f6a39fe-817f-42a6-99d0-a0801d2722bf',
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  0f6a39fe-817f-42a6-99d0-a0801d2722bf


## Initialize custom objects

In [9]:
data_loader = ACSIncomeDataset(state=['CA'], year=2018, with_nulls=False,
                               subsample_size=15_000, subsample_seed=42)
data_loader.X_data.head()

,SCHL,COW,MAR,OCCP,POBP,RELP,SEX,RAC1P,AGEP,WKHP
0,13,1,1,8030,247,0,1,6,47,40.0
1,18,3,1,930,6,1,2,1,55,40.0
2,16,1,5,4760,6,2,2,1,20,30.0
3,21,1,1,5240,150,0,2,1,31,40.0
4,19,1,1,8990,6,2,1,1,37,42.0


In [10]:
data_loader.X_data.shape

(15000, 10)

## Run experiment iterations

### Experiment iteration 1

In [13]:
tuned_params_filenames = [
    'tuning_results_Folktables_CA_2018_Income_alpha_0.0_20240224__154709.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', META_EXPERIMENT_NAME,
                                      EXPERIMENT_NAME, '50', tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]

In [14]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)

In [ ]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
                                   # with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   dataset_name='ACSIncomeDataset',
                                   verbose=True)

2024-02-24 17:01:09 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'experiment_iteration': 'Exp_iter_1',
 'fair_intervention_params_lst': '[0.0]',
 'model_init_seed': 100,
 'session_uuid': '0f6a39fe-817f-42a6-99d0-a0801d2722bf'}




Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-02-24 17:01:10 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-02-24 17:01:10 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


intervention_param:  0.0
Skip preprocessing
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__SCHL_1', 'cat__SCHL_10', 'cat__SCHL_11', 'cat__SCHL_12',
       'cat__SCHL_13', 'cat__SCHL_14', 'cat__SCHL_15', 'cat__SCHL_16',
       'cat__SCHL_17', 'cat__SCHL_18',
       ...
       'cat__RELP_2', 'cat__RELP_3', 'cat__RELP_4', 'cat__RELP_5',
       'cat__RELP_6', 'cat__RELP_7', 'cat__RELP_8', 'cat__RELP_9', 'num__AGEP',
       'num__WKHP'],
      dtype='object', length=728)
Top indexes of an X_test in the current base flow dataset:  Int64Index([10155, 11689, 12599, 12193,  8678,  8217,  4670, 12087,  5235,
             4189,  7278, 10642,  5284,  7002, 14642, 10594,  7701,  8686,
             8665,  6253],
           dtype='int64')
Top indexes of an y_test in the current base flow dataset:  Int64Index([10155, 11689, 12599, 12193,  8678,  8217,  4670, 12087,  5235,
             4189,  7278, 10642,  5284,  7002, 14642, 10594,  7701,  8686,
             8665,  6253],
           dtype='i

Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/10 [00:00<?, ?it/s]

### Experiment iteration 2

In [15]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_CA_2018_Income_alpha_0.0_20240224__154709.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', META_EXPERIMENT_NAME, EXPERIMENT_NAME, '50', tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)

In [16]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
                                   # with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   dataset_name='ACSIncomeDataset',
                                   verbose=True)

2024-02-24 11:37:31 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 200,
 'experiment_iteration': 'Exp_iter_2',
 'fair_intervention_params_lst': '[0.0]',
 'intervention_param': 0.0,
 'model_init_seed': 200,
 'run_start_date_time': datetime.datetime(2024, 2, 24, 16, 17, 2, 907105, tzinfo=datetime.timezone.utc),
 'session_uuid': 'd9a91e93-d876-40fe-98be-41ec023e789f'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-02-24 11:37:32 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-02-24 11:37:32 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


intervention_param:  0.0
Skip preprocessing
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__SCHL_1', 'cat__SCHL_10', 'cat__SCHL_11', 'cat__SCHL_12',
       'cat__SCHL_13', 'cat__SCHL_14', 'cat__SCHL_15', 'cat__SCHL_16',
       'cat__SCHL_17', 'cat__SCHL_18',
       ...
       'cat__RELP_2', 'cat__RELP_3', 'cat__RELP_4', 'cat__RELP_5',
       'cat__RELP_6', 'cat__RELP_7', 'cat__RELP_8', 'cat__RELP_9', 'num__AGEP',
       'num__WKHP'],
      dtype='object', length=716)
Top indexes of an X_test in the current base flow dataset:  Int64Index([ 6043,  3745,  5159,  7241,  7820,  3695, 11501, 11432,  1163,
             8994,  7972,  2554,  9884,  2008,  6884, 11995,  5200,  4649,
            10244, 13775],
           dtype='int64')
Top indexes of an y_test in the current base flow dataset:  Int64Index([ 6043,  3745,  5159,  7241,  7820,  3695, 11501, 11432,  1163,
             8994,  7972,  2554,  9884,  2008,  6884, 11995,  5200,  4649,
            10244, 13775],
           dtype='i

Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/101 [00:00<?, ?it/s]

### Experiment iteration 3

In [17]:
# Configs for an experiment iteration
exp_iter_num = 3
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_CA_2018_Income_alpha_0.0_20240224__154709.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', META_EXPERIMENT_NAME, EXPERIMENT_NAME, '50', tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)

In [18]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
                                   # with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   dataset_name='ACSIncomeDataset',
                                   verbose=True)

2024-02-24 11:56:31 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 300,
 'experiment_iteration': 'Exp_iter_3',
 'fair_intervention_params_lst': '[0.0]',
 'intervention_param': 0.0,
 'model_init_seed': 300,
 'run_start_date_time': datetime.datetime(2024, 2, 24, 16, 37, 32, 273935, tzinfo=datetime.timezone.utc),
 'session_uuid': 'd9a91e93-d876-40fe-98be-41ec023e789f'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-02-24 11:56:31 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-02-24 11:56:31 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


intervention_param:  0.0
Skip preprocessing
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__SCHL_1', 'cat__SCHL_10', 'cat__SCHL_11', 'cat__SCHL_12',
       'cat__SCHL_13', 'cat__SCHL_14', 'cat__SCHL_15', 'cat__SCHL_16',
       'cat__SCHL_17', 'cat__SCHL_18',
       ...
       'cat__RELP_2', 'cat__RELP_3', 'cat__RELP_4', 'cat__RELP_5',
       'cat__RELP_6', 'cat__RELP_7', 'cat__RELP_8', 'cat__RELP_9', 'num__AGEP',
       'num__WKHP'],
      dtype='object', length=722)
Top indexes of an X_test in the current base flow dataset:  Int64Index([ 4329,   723,  8157,  8934,  5880,  1877,  9980,  9115, 13634,
            12168,  6957,  9083, 14260,  5349,  7879,  3078, 10032, 13643,
            12025,  7489],
           dtype='int64')
Top indexes of an y_test in the current base flow dataset:  Int64Index([ 4329,   723,  8157,  8934,  5880,  1877,  9980,  9115, 13634,
            12168,  6957,  9083, 14260,  5349,  7879,  3078, 10032, 13643,
            12025,  7489],
           dtype='i

Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/101 [00:00<?, ?it/s]

### Experiment iteration 4

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 4
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_CA_2018_Income_alpha_0.0_20240224__154709.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', META_EXPERIMENT_NAME, EXPERIMENT_NAME, '50', tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)

In [ ]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
                                   # with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   dataset_name='ACSIncomeDataset',
                                   verbose=True)

### Experiment iteration 5

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 5
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_CA_2018_Income_alpha_0.0_20240224__154709.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', META_EXPERIMENT_NAME, EXPERIMENT_NAME, '50', tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)

In [ ]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
                                   # with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   dataset_name='ACSIncomeDataset',
                                   verbose=True)

### Experiment iteration 6

In [11]:
# Configs for an experiment iteration
exp_iter_num = 6
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_CA_2018_Income_alpha_0.0_20240224__154709.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', META_EXPERIMENT_NAME, EXPERIMENT_NAME, '50', tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)

In [ ]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
                                   # with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   dataset_name='ACSIncomeDataset',
                                   verbose=True)

2024-02-24 17:03:27 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 600,
 'experiment_iteration': 'Exp_iter_6',
 'fair_intervention_params_lst': '[0.0]',
 'model_init_seed': 600,
 'session_uuid': '0f6a39fe-817f-42a6-99d0-a0801d2722bf'}




Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-02-24 17:03:27 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-02-24 17:03:27 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


intervention_param:  0.0
Skip preprocessing
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__SCHL_1', 'cat__SCHL_10', 'cat__SCHL_11', 'cat__SCHL_12',
       'cat__SCHL_13', 'cat__SCHL_14', 'cat__SCHL_15', 'cat__SCHL_16',
       'cat__SCHL_17', 'cat__SCHL_18',
       ...
       'cat__RELP_2', 'cat__RELP_3', 'cat__RELP_4', 'cat__RELP_5',
       'cat__RELP_6', 'cat__RELP_7', 'cat__RELP_8', 'cat__RELP_9', 'num__AGEP',
       'num__WKHP'],
      dtype='object', length=721)
Top indexes of an X_test in the current base flow dataset:  Int64Index([ 2649,  1964, 14464,  4377, 14152,  5747,  1529, 10243, 12578,
             9794,  7142, 14347,  6387,  2917,   317,  9085,  5821, 11007,
             8142,  3719],
           dtype='int64')
Top indexes of an y_test in the current base flow dataset:  Int64Index([ 2649,  1964, 14464,  4377, 14152,  5747,  1529, 10243, 12578,
             9794,  7142, 14347,  6387,  2917,   317,  9085,  5821, 11007,
             8142,  3719],
           dtype='i

Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/10 [00:00<?, ?it/s]